In [1]:
# Load modules
from dash import Dash, dcc, html, Input, Output,callback, State
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import plotly.express as px
import dash

import altair as alt
import numpy as np
import pandas as pd

murder = pd.read_csv("database.csv/database.csv")

#remove space to conduct query
murder.columns =['Record_ID', 'Agency_Code', 'Agency_Name', 'Agency_Type', 'City',
       'State', 'Year', 'Month', 'Incident', 'Crime_Type', 'Crime_Solved',
       'Victim_Sex', 'Victim_Age', 'Victim_Race', 'Victim_Ethnicity',
       'Perpetrator_Sex', 'Perpetrator_Age', 'Perpetrator_Race',
       'Perpetrator_Ethnicity', 'Relationship', 'Weapon', 'Victim_Count',
       'Perpetrator_Count', 'Record_Source']

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
#Map state names to state codes (Needed for plotting)
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

#The actual mapping 
murder["State_Code"] = murder["State"].replace(us_state_to_abbrev, inplace = False)

def df_text_generate(df_group,var):
    text_dic = {}
    for idx in df_group.index:
        if idx not in text_dic:
            temp = var.replace('_',' ') + '<br>'
            for i in range(0,len(df_group.loc[idx][var])):
                temp += df_group.loc[idx][var][i] + ': ' + str(df_group.loc[idx]['Record_ID'][i]) + '<br>'
            text_dic[idx] = temp
    return text_dic

def plot_text_generate(df):
    murder_group1 = df.groupby(['State_Code','Victim_Sex']).count().reset_index(level=1)
    text_lst1 = df_text_generate(murder_group1,'Victim_Sex')
    murder_group2 = df.groupby(['State_Code','Perpetrator_Sex']).count().reset_index(level=1)
    text_lst2 = df_text_generate(murder_group2,'Perpetrator_Sex')
    
    df_trial = df.groupby('State_Code').count().reset_index(level=0)
    plot_text_lst = [x+y for x,y in zip(list(text_lst1.values()), list(text_lst2.values()))]
    df_trial['temp_text'] = plot_text_lst
    df_trial["Record_ID"] = df_trial["Record_ID"].astype("str")
    df_trial['plot_text'] = 'State Code: ' + df_trial['State_Code'] + '<br>Record Number: ' + df_trial['Record_ID'] + '<br>' + df_trial['temp_text']
    return df_trial

In [3]:
# Instantiate the app
#Jupyter dash
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

server = app.server

# CSS Styles
css_dd = {
    "font-size": "smaller",
}

css_list = {
    "column-count": 2,
}

css_sources = {
    "font-size": "xx-small",
}

YEARS = [2010, 2011, 2012, 2013, 2014]
WEAPONS = [w.strip() for w in list(murder['Weapon'].dropna().unique())]
#### Set Layout
app.layout = dbc.Container([
        
    html.H1("Dashboard of US Murder Cases by Cities"),
    html.P("some sentence"),
    
    html.Div(
    [
        dbc.Row(
            [
                dbc.Col([html.Label("Year: "),
                       dcc.Slider(id="years_slider",
                                  min=min(YEARS),
                                  max=max(YEARS),
                                  step=1,
                                  value=min(YEARS),
                                  marks={str(year): {"label": str(year),
                                                     "style": {"color": "#7fafdf"}}
                                         for year in YEARS})]),
                dbc.Col([html.Label("Weapon: "),
                        dbc.DropdownMenu(
                            style=css_dd,
                            children=[
                                dcc.Checklist(
                                    id='weapons_checklist',
                                    options=WEAPONS,
                                    value=[],
                                    labelStyle = {'display': 'block'},
                                    className="css_list",)
                            ],
                            label='choices',
                        )])
            ]),
        
        #map graph
        dbc.Row(
            [
                dcc.Graph(
                    id="map_graph",
                )
            ]),
    ])
])

#### Callback 
@app.callback(
    Output("map_graph", "figure"),
    Input("years_slider", "value"),
    Input("weapons_checklist", "value"),
)

def plot_map(years_slider,weapons_checklist):
    #filter
    murder_selected = murder.loc[(murder['Year']==years_slider) & 
                                (murder['Weapon'].isin(weapons_checklist))]
    murder_trial = plot_text_generate(murder_selected)
    
    #graph
    base = px.choropleth(murder_trial,
                         locations='State_Code', 
                         locationmode="USA-states", 
                         scope="usa",
                         color='Record_ID',color_continuous_scale="Viridis_r", 
                        )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return base
    
    

if __name__ == '__main__':
    app.server.run(port=8000, host='127.0.0.1')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Mar/2023 18:09:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2023 18:09:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2023 18:09:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2023 18:09:15] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Mar/2023 18:09:15] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2023 18:09:15] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2023 18:09:15] "GET /_favicon.ico?v=2.8.1 HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2023 18:09:15] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2023 18:09:19] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Mar/2023 18:09:21] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Mar/2023 18:09:21] "POST /_dash-update-component HTTP/1.1" 50

In [7]:
murder_selected = murder.loc[(murder['Year']==2010)]

In [8]:
#The actual mapping 
murder["State_Code"] = murder["State"].replace(us_state_to_abbrev, inplace = False)

def df_text_generate(df_group,var):
    text_dic = {}
    for idx in df_group.index:
        if idx not in text_dic:
            temp = var.replace('_',' ') + '<br>'
            if type(df_group.loc[idx][var]) == str:
                temp += df_group.loc[idx][var] + ': ' + str(df_group.loc[idx]['Record_ID']) + '<br>'
            else:
                for i in range(0,len(df_group.loc[idx][var])):
                    temp += df_group.loc[idx][var][i] + ': ' + str(df_group.loc[idx]['Record_ID'][i]) + '<br>'
            text_dic[idx] = temp
    return text_dic

def plot_text_generate(df):
    murder_group1 = df.groupby(['State_Code','Victim_Sex']).count().reset_index(level=1)
    text_lst1 = df_text_generate(murder_group1,'Victim_Sex')
    murder_group2 = df.groupby(['State_Code','Perpetrator_Sex']).count().reset_index(level=1)
    text_lst2 = df_text_generate(murder_group2,'Perpetrator_Sex')
    
    df_trial = df.groupby(['State_Code','State']).count().reset_index(level=1).reset_index(level=0)
    plot_text_lst = [x+y for x,y in zip(list(text_lst1.values()), list(text_lst2.values()))]
    df_trial['temp_text'] = plot_text_lst
    df_trial["Record_ID"] = df_trial["Record_ID"].astype("str")
    df_trial['plot_text'] = 'State: ' + df_trial['State'] + '<br>' + df_trial['temp_text']
    return df_trial

In [9]:
pop = data.population_engineers_hurricanes()
pop.head()

NameError: name 'data' is not defined

In [10]:
us_state_to_id = dict(zip(pop['state'], pop['id']))
murder["id"] = murder["State"].replace(us_state_to_id, inplace = False)

NameError: name 'pop' is not defined

In [11]:
murder_selected = murder.loc[(murder['Year']==2013) & (murder['Weapon']=='Gun')]

In [12]:
murder_trial = plot_text_generate(murder_selected)

In [13]:
state_selected = list(murder_trial['State_Code'])

In [14]:
Month_sort = ['January','February','March','April','May','June','July','August','September','October','November','December']
gender = ['Female','Male','Unknown']
month_gender = [(gen,mon) for gen in gender for mon in Month_sort]

In [15]:
def line_graph_sta(df_selected):
    df_filter = df_selected.groupby(['Month','Perpetrator_Sex']).count().reset_index(level=0).reset_index(level=0).groupby(['Perpetrator_Sex','Month']).sum().iloc[:,:1]
    df_month_pair = list(df_filter.index)
    miss_month_pair = [mp for mp in month_gender if mp not in df_month_pair]
    index=pd.MultiIndex.from_tuples(miss_month_pair, names=['Perpetrator_Sex', 'Month'])
    new=pd.DataFrame([{'Record_ID':0}],index=index)
    return pd.concat([new,df_filter]).groupby(['Month','Perpetrator_Sex']).sum().reindex(Month_sort,level=0).reset_index(level=0).reset_index(level=0)

In [23]:
a=line_graph_sta(murder_selected)

In [24]:
fig1 = px.line(a,
              x="Month", y="Record_ID", color='Perpetrator_Sex')
fig1.show()

In [25]:
set(murder['Perpetrator_Race'])

{'Asian/Pacific Islander',
 'Black',
 'Native American/Alaska Native',
 'Unknown',
 'White'}

In [26]:
murder_selected

,Record_ID,Agency_Code,Agency_Name,Agency_Type,City,State,Year,Month,Incident,Crime_Type,...,Perpetrator_Sex,Perpetrator_Age,Perpetrator_Race,Perpetrator_Ethnicity,Relationship,Weapon,Victim_Count,Perpetrator_Count,Record_Source,State_Code
595912,595913,AR03501,Pine Bluff,Municipal Police,Jefferson,Arkansas,2013,May,1,Murder or Manslaughter,...,Male,21,Black,Unknown,Unknown,Gun,0,0,FBI,AR
596072,596073,AZ00717,Mesa,Municipal Police,Maricopa,Arizona,2013,January,1,Murder or Manslaughter,...,Unknown,0,Unknown,Unknown,Unknown,Gun,0,0,FBI,AZ
596172,596173,AZ00723,Phoenix,Municipal Police,Maricopa,Arizona,2013,June,5,Murder or Manslaughter,...,Male,32,White,Not Hispanic,Unknown,Gun,0,0,FBI,AZ
596233,596234,AZ00723,Phoenix,Municipal Police,Maricopa,Arizona,2013,October,14,Murder or Manslaughter,...,Male,54,White,Not Hispanic,Unknown,Gun,0,0,FBI,AZ
596302,596303,AZ01001,South Tucson,Municipal Police,Pima,Arizona,2013,August,1,Murder or Manslaughter,...,Unknown,0,Unknown,Unknown,Unknown,Gun,0,0,FBI,AZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609917,609918,WA03100,Snohomish County,Sheriff,Snohomish,Washington,2013,March,1,Murder or Manslaughter,...,Female,32,White,Not Hispanic,Acquaintance,Gun,0,0,FBI,WA
609944,609945,WA03204,Spokane,Municipal Police,Spokane,Washington,2013,October,1,Murder or Manslaughter,...,Male,34,White,Not Hispanic,Unknown,Gun,0,0,FBI,WA
609982,609983,WASPD00,Seattle,Municipal Police,King,Washington,2013,March,1,Murder or Manslaughter,...,Male,19,Black,Unknown,Unknown,Gun,0,0,FBI,WA
610091,610092,WIMPD00,Milwaukee,Municipal Police,Milwaukee,Wisconsin,2013,April,2,Murder or Manslaughter,...,Male,41,Black,Unknown,Girlfriend,Gun,0,0,FBI,WI


In [27]:
a= murder_selected.groupby(['Perpetrator_Race','Perpetrator_Sex']).count().reset_index(level=0).reset_index(level=0)

In [28]:
fig2 = px.bar(a, x="Perpetrator_Race", y="Record_ID", color="Perpetrator_Sex")
fig2.show()

In [30]:
fig1 & fig2

TypeError: unsupported operand type(s) for &: 'Figure' and 'Figure'

## Altair

In [ ]:
import altair as alt
from vega_datasets import data

In [6]:
murder_selected = murder.loc[(murder['Year']==2010)]
murder_trial = murder_selected.groupby(["id",'State']).count().reset_index()

KeyError: 'id'

In [108]:
df_filter = murder_selected.groupby(['State','Month','Perpetrator_Sex']).count().reset_index().groupby(['Perpetrator_Sex','Month','State']).sum().iloc[:,:1]

In [109]:
df_month_pair = list(df_filter.index)

In [110]:
month_gender_state = [(gen,mon,state) for gen in gender for mon in Month_sort for state in set(murder['State'])]

In [111]:
miss_month_pair = [mp for mp in month_gender_state if mp not in df_month_pair]

In [112]:
index=pd.MultiIndex.from_tuples(miss_month_pair, names=['Perpetrator_Sex', 'Month','State'])

In [113]:
new=pd.DataFrame([{'Record_ID':0}],index=index)

In [114]:
tttt=pd.concat([new,df_filter]).groupby(['Month','Perpetrator_Sex','State']).sum().reindex(Month_sort,level=0).reset_index()

In [115]:
tttt["id"] = tttt["State"].replace(us_state_to_id, inplace = False)

In [117]:
t2=tttt.groupby(['id','State']).sum().reset_index()

In [254]:
import altair as alt
from vega_datasets import data


#state_pop = data.population_engineers_hurricanes()[['state', 'id', 'population']]
state_map = alt.topo_feature(data.us_10m.url, 'states')

click = alt.selection_multi(fields=['State'])

choropleth = (alt.Chart(state_map).mark_geoshape().transform_lookup(
    lookup='id',
    from_=alt.LookupData(t2, 'id', list(t2.columns)))
.encode(
    color='Record_ID:Q',
    opacity=alt.condition(click, alt.value(1), alt.value(0.2)),
    tooltip=['Record_ID:Q'])
.add_selection(click)
.project(type='albersUsa').properties(
    width=500,
    height=700
))

'''lines = (
    alt.Chart(
        tttt,).mark_line().encode(
    x=alt.X('Month',sort=Month_sort),
    y='sum(Record_ID):Q',
    color='Perpetrator_Sex',
    tooltip=['Month', 'Perpetrator_Sex', 'sum(Record_ID)'])
.transform_filter(click).properties(
    width=800,
    height=300
).interactive())'''

highlight = alt.selection(type='single', on='mouseover', fields=['Month'], nearest=True)
base=alt.Chart(tttt).encode(x=alt.X('Month',sort=Month_sort),
                           y='sum(Record_ID):Q',
                           color='Perpetrator_Sex').transform_filter(click)
lines=base.mark_line().encode()
selectors = base.mark_point().encode(opacity=alt.value(0),).add_selection(highlight)
stpoints = base.mark_point(size=40).encode(opacity=alt.condition(highlight, alt.value(1), alt.value(0)))
text = base.mark_text(align='left', dx=5, dy=-5,size=18).encode(text=alt.condition(highlight, 'sum(Record_ID):Q', alt.value(' ')))
rules = base.mark_rule().encode().transform_filter(highlight)

all_g = alt.layer(lines,selectors,stpoints,text,rules).properties(width=200, height=300).interactive()


bar = (alt.Chart(a,).mark_bar().encode(
    x=alt.X('Perpetrator_Race'),
    y='sum(Record_ID):Q',
    color='Perpetrator_Sex',
    tooltip=['Perpetrator_Race', 'Perpetrator_Sex', 'sum(Record_ID)'])
.transform_filter(click).properties(
    width=200,
    height=300
).interactive())


(choropleth | (all_g & bar))

alt.HConcatChart(...)

In [250]:
a= murder_selected.groupby(['Perpetrator_Race','Perpetrator_Sex','State']).count().reset_index(level=0).reset_index()

In [251]:
a["id"] = a["State"].replace(us_state_to_id, inplace = False)

In [252]:
a

,Perpetrator_Sex,State,Perpetrator_Race,Record_ID,Agency_Code,Agency_Name,Agency_Type,City,Year,Month,...,Victim_Ethnicity,Perpetrator_Age,Perpetrator_Ethnicity,Relationship,Weapon,Victim_Count,Perpetrator_Count,Record_Source,State_Code,id
0,Female,Alaska,Asian/Pacific Islander,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2
1,Female,California,Asian/Pacific Islander,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,6
2,Female,Florida,Asian/Pacific Islander,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,12
3,Female,Nevada,Asian/Pacific Islander,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,32
4,Female,New Mexico,Asian/Pacific Islander,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,Male,Washington,White,104,104,104,104,104,104,104,...,104,104,104,104,104,104,104,104,104,53
324,Male,West Virginia,White,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,54
325,Male,Wisconsin,White,59,59,59,59,59,59,59,...,59,59,59,59,59,59,59,59,59,55
326,Male,Wyoming,White,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,56


In [240]:
bar = (alt.Chart(a,).mark_bar().encode(
    x=alt.X('Perpetrator_Race'),
    y='sum(Record_ID):Q',
    color='Perpetrator_Sex',
    tooltip=['Perpetrator_Race', 'Perpetrator_Sex', 'sum(Record_ID)'])
.properties(
    width=800,
    height=300
).interactive())

In [241]:
bar

alt.Chart(...)